<img src="./img/oscon.png" width="400">

# Session 4: TensorBoard Visualization

In this session, we explore the use of TensorBoard, to visualize the training + computation graph of the simple softmax classifier trained on MNIST we saw in previous sessions. 

## Reminder: MNIST softmax classifier

In [ ]:
from __future__ import print_function

import tensorflow as tf
import sys,os
from tensorflow.examples.tutorials.mnist import input_data
DATA_DIR = os.path.join(os.environ["HOME"], "data") if not 'win' in sys.platform else "c:\\tmp\\data"

NUM_STEPS = 1000
MINIBATCH_SIZE = 100

In [ ]:
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.add(tf.matmul(x, W),b)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true))

gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
# Train
sess.run(tf.global_variables_initializer())

for _ in range(NUM_STEPS):
    batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
    sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})


In [ ]:
# Test
ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})

print ("Accuracy: {:.4}%".format(ans*100))
sess.close()

## Adding summaries

We'd like to visualize our TensorFlow graph, and plot metrics from the training process. For graph visualization, we add variable name scopes to define a hierarchy on nodes in the graph that is used by TensorBoard when displaying the graph. Summary operations are attached to nodes in the graph we are interested in plotting. 

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.name_scope('input_x'):
    x = tf.placeholder(tf.float32, [None, 784])
with tf.name_scope('input_label'):
    y_true = tf.placeholder(tf.float32, [None, 10]) 
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

In [ ]:
with tf.name_scope('weights'):    
    W = tf.Variable(tf.zeros([784, 10]))
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(W)
      tf.summary.scalar('mean', mean)
      tf.summary.histogram('histogram', W)
with tf.name_scope('biases'):    
    b = tf.Variable(tf.zeros([10]))
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(b)
      tf.summary.scalar('mean', mean)
      tf.summary.histogram('histogram', b)

with tf.name_scope('Wx_b'):     
    y_pred = tf.add(tf.matmul(x, W),b)
    tf.summary.histogram('Wx_b', y_pred)

In [ ]:
with tf.name_scope('loss'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true))
    tf.summary.scalar('loss', cross_entropy)
    
with tf.name_scope('train'):
    gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
with tf.name_scope('correct_pred'):
        correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
with tf.name_scope('accuracy'):
  accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))


We next merge all the summaries into one op, generating a serialized object with all summary data at a given step. This serialized object is passed to a FileWriter, which writes the data to disk. FileWriter takes a logdir, which is the path to the folder we save all data. If given a TensorFlow graph object, FileWriter will also create a very useful visualization of our computation graph.

In [ ]:
# Merge all the summaries and write them out to LOG_DIR
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(os.path.join(DATA_DIR,"logs\\demo\\train"), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(DATA_DIR,"logs\\demo\\test"))

In [ ]:
# Train
sess.run(tf.global_variables_initializer())

for i in range(NUM_STEPS):

    batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
    summary, _ = sess.run([merged, gd_step], feed_dict={x: batch_xs, y_true: batch_ys})
    train_writer.add_summary(summary, i)
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict={x: data.test.images, y_true: data.test.labels})
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
train_writer.close()
test_writer.close()        

Run the command line: tensorboard --logdir=demo:DATA_DIR [replace DATA_DIR with your path]

Then, in your browser, go to localhost:6006 (or the address printed in the console)

Note: TensorBoard recursively walks the directory tree rooted at logdir looking for subdirectories that contain tfevents data. 

## Quick hands-on

Take TensorBoard for a spin:

* Add some more types of summaries (you'll need to use the suitable TensorFlow ops that you saw earlier!):
    * Max
    * Min
    * Standard deviation
    
* Run and check out results. To avoid overwriting of existing data, add another subdirectory to the log directory given to FileWriter. For example, use tf.summary.FileWriter(os.path.join(DATA_DIR,"logs\\ex1\\train"), sess.graph).


In [ ]:
# %load ..\solutions\tfboard_ex1.py    